<a href="https://colab.research.google.com/github/evan15900/Lee113/blob/main/EX05_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.7 MB/s eta 0:00:00


In [2]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_Channel_access_token'))
handler = WebhookHandler(userdata.get('LINE_Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
          #貼圖
          reply = StickerMessage(package_id='6362', sticker_id='11087939')
        elif action == 'image':
          #圖片
          reply = ImageMessage(
              original_content_url='https://i.imgur.com/NLORoDg.jpg',
              preview_image_url='https://i.imgur.com/NLORoDg.jpg'
          )
        elif action == 'video':
          #影片
          reply = VideoMessage(
              original_content_url='https://cdn.pixabay.com/video/2025/04/09/270940_large.mp4',
              preview_image_url='https://cdn.pixabay.com/photo/2016/11/06/17/11/zermatt-1803481_1280.jpg'
          )
        elif action == 'location':
          #位置
          reply = LocationMessage(
              title='國立中山大學',
              address='高雄市鼓山區蓮海路70號',
              latitude=22.628439826342117,
              longitude=120.26471679137201
          )
        else:
          #文字
          response = ask_gemini(action)
          reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5c13-35-196-61-39.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:52:00] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:52:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:52:09] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:52:31] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:52:50] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:53:02] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:53:28] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 16:53:56] "POST /callback HTTP/1.1" 200 -
